# Python Cleaning Data

This chapter, you will understand how to prepare your data for analysis and diagnose data for problems.

### DataFrame Overview Methods

`df.head()`, `df.tail()`

Use it to have quick look on data.

`df.shape`

You can check many columns and rows in your dasta.

`df.columns`

You can see what's the column name.

`df.info()`

You can check column types and non-null values per column.

In [ ]:
import pandas as pd

pm_2_5_file_path = './res/observation_pm_2_5.csv'

# Some rows has value 'NR', we treat it as null
df = pd.read_csv(pm_2_5_file_path, na_values='NR')

print(df.head(2))

print(df.tail(2))

print(df.shape)

print(df.columns)

print(df.info())

### DataFrame Exploratory Methods

`df.value_counts()`

Frequency counts for categorical data.

`df.describe()`

Have a quick understanding of your column values distribution.

In [ ]:
print(df['Date'].value_counts().head())

print(df['Observation Item'].value_counts().head())

print(df['0'].describe())

pm_2_5_df = df[df.iloc[:, 1] == 'PM2.5']

print(pm_2_5_df.describe())

### Q: The world countreis life expectancy number is in path `./res/g1800.csv`, please read it to DataFrame `g1800_df` with Pandas and have a look on data.

Hint: When you call method `info()`, please only select first 100 columns, otherwise you will not get non-null count

In [ ]:
## Write your code here


### Visually Explore Data

##### Histogram

Use it to look at frequency.

In [ ]:
import matplotlib.pyplot as plt

pm_2_5_df['0'].plot(kind='hist', bins=20)

plt.show()

##### Box Plots

Visualize basic summary statistics.

* Outliers
* Min/Max
* 25th,50th,75th percentiles

After you find outliers values, you can remove those rows(call `df.drop(indice_name_list)`) if you like.

In [ ]:
import matplotlib.pyplot as plt

df.boxplot(column=['0'], by='Observation Item', figsize=(18, 6))

plt.show()

##### Scatter Plots

Use it to see correlation between two columns

In [ ]:
import matplotlib.pyplot as plt

o3_df = df[df.iloc[:, 1] == 'O3']
pm10_df = df[df.iloc[:, 1] == 'PM10']

print(pm_2_5_df.head())
print(o3_df.head())
print(pm10_df.head())

plt.scatter(pm_2_5_df['0'], o3_df['0'])
plt.xlabel('pm 2.5')
plt.ylabel('O3')
plt.show()

plt.scatter(pm_2_5_df['0'], pm10_df['0'])
plt.xlabel('pm 2.5')
plt.ylabel('PM 10')
plt.show()

### Q: Please visualize DataFrame `g1800_df` and see what's your finding.

* See histgram chart using data in year 1800.
* See scatter chart using data in year 1800 and in year 1899, if they have correlation?

In [ ]:
# Write your code here


### Tidy Data

Tidy data makes it better for analysis and easier to fix column data problems.

##### Priciples of Tidy Data

* Columns represents separate variables
* Row represents individual observations
* Observational units forms tabls

Use `pd.melt()` to reshape DataFrame. Put column names which to be stayed in columns into argument `id_vars`, others will be melted into columns `variable`, `value`. You can rename these two column names.

In [ ]:
import pandas as pd

print(df.head())

melt_df = pd.melt(frame=df, id_vars=['Date', 'Observation Item'], var_name='hour', value_name='value')

print(melt_df.head(50))

### Q: The Format of DataFrame `g1800_df` is not good to analyze, please convert into tidy data.

In [ ]:
# Write your code here


### Pivot Data

It is the opposite of `melting`. In melting, columns are turned into rows. In Pivoting, rows are turned into columns.

It could convert analysis friendly shape to report friendly shape.

Use `pd.pivot_table()` to reshape DataFrame. Put variables which to be kept in columns in `index`, fill `columns` with other variables, and fill `values` with values

`You will not able to pivot data if you have duplicated entries`.

In [ ]:
import pandas as pd

# Column order of pivot table will be sorted by column, convert type to number to have expected order
melt_df['hour'] = melt_df['hour'].apply(pd.to_numeric)

print(melt_df.head())

pivot_df = melt_df.pivot_table(index=['Date', 'Observation Item'], columns='hour', values='value')

print(pivot_df.head())

pivot_df = pivot_df.reset_index()

print(pivot_df.head())

You can use argument `aggfunc` to remove duplicates or calculate aggregate result, available functions are `np.sum`, `np.mean`, etc.

In [ ]:
import numpy as np
import pandas as pd

print(melt_df.head())

pivot_agg_df = melt_df.pivot_table(index=['Date', 'Observation Item'], values='value', aggfunc=np.mean)

print(pivot_agg_df.head())

### DataFrame Operation

##### Concatenation

`pd.concat` is used for concatenate multiple dataframes.

In [ ]:
import numpy as np
import pandas as pd

au_2018_df = pd.DataFrame(np.array([['2018-01-01', '13000'], ['2018-01-02', '13645']]))
au_2018_df.columns = ['year', 'active_users']
print(au_2018_df)

au_2017_df = pd.DataFrame(np.array([['2017-01-01', '10329'], ['2017-01-02', '10986']]))
au_2017_df.columns = ['year', 'active_users']
print(au_2017_df)

au_concat_df = pd.concat([au_2018_df, au_2017_df])

print(au_concat_df.shape)
print(au_concat_df.head())

You can concatenate data for different axis.

In [ ]:
import numpy as np
import pandas as pd

print(au_2018_df)

ru_2018_df = pd.DataFrame(np.array([['2018-01-01', '256'], ['2018-01-02', '283']]))
ru_2018_df.columns = ['year', 'register_users']
print(ru_2018_df)

au_ru_concat_df = pd.concat([au_2018_df, ru_2018_df], axis=1)
print(au_ru_concat_df.head())

##### Globbing

you can use `pd.blob()` if you have many files need to be concatenated.

Use Pattern match for file names.

Wildcards:

* `*`: match any csv files: *.csv 
* `?`: match any single characters(A-z, 0-9): file_?.csv

In [ ]:
import glob

csv_files = glob.glob('./res/*.csv')
print(csv_files)

yearly_registers_files = glob.glob('./res/yearly_registers*')
print(yearly_registers_files)

yearly_registers_files = glob.glob('./res/yearly_registers.???')
print(yearly_registers_files)

With `glob`, you can concatenate dataframes by following way.

In [ ]:
import glob
import pandas as pd

registers_files = glob.glob('./res/monthly_registers_*.csv')


register_dfs = []

for file in registers_files:
    print(file)
    register_df = pd.read_csv(file)
    register_dfs.append(register_df)

register_concat_df = pd.concat(register_dfs)
print(register_concat_df.head(15))

### Q: We have year 1800, 1900, 2000 life expectancy number in different files, `./res/g1800.csv`,  `./res/g1900.csv`,  `./res/g2000.csv`, please tidy those data and concatenate it into DataFrame `life_expectancy_df`.

In [ ]:
# Write your code here


##### Merging Data

Similar to `join` operation in SQL.

Use `pd.merge()` to merge DataFrames.

In [ ]:
import pandas as pd

countries_registers_df = pd.read_csv('./res/country_registers_2017.csv')
countries_df = pd.read_csv('./res/countries.csv')

print(countries_registers_df.head())
print(countries_df.head())

merged_df = pd.merge(left=countries_registers_df, right=countries_df, left_on='countryid', right_on='countryid')
print(merged_df.head())

##### Converting Data Type

You can convert data type to your expected one.

In [ ]:
import pandas as pd

registers_2017_df = pd.read_csv('./res/monthly_registers_2017.csv')
print(registers_2017_df.head())
print(registers_2017_df.dtypes)

registers_2017_df['amount'] = registers_2017_df['amount'].astype(float)
print(registers_2017_df.dtypes)

If a column is a category field, you can use `category` type to save memory size.

In [ ]:
import pandas as pd

pm_2_5_file_path = './res/observation_pm_2_5.csv'

df = pd.read_csv(pm_2_5_file_path)

print(df.dtypes.head(5))
print(df.info())

df['Observation Item'] = df['Observation Item'].astype('category')

print(df.dtypes.head(5))
print(df.info())

If you column contains other type instead of numeric, it will show error.

In [ ]:
print(df['0'].head(18))

print(df['0'].value_counts().head())

df['0'] = pd.to_numeric(df['0'])

Add argument `errors='coerce'` can solve this problem.

The value cannot be converted into number will be ignored.

In [ ]:
df['0'] = pd.to_numeric(df['0'], errors='coerce')

print(df['0'].value_counts().head())

##### Parsing string with Regular Expression

You can search online for Regular Expression Syntax, it will be not covered here.

Use `.str.match()` to check if field matchs a pattern.

In [ ]:
print(df['Observation Item'].head(18))

is_match_pm = df['Observation Item'].str.match('PM.+')

print(df[is_match_pm].head())

Use `.str.extract()` to extract match string, and use `.str.extractall` to extract match string groups

In [ ]:
print(df['Observation Item'].head(18))

print(df['Observation Item'].str.extract('PM([\d.]+)', expand=True).head(18))
print(df['Observation Item'].str.extractall('(.)').head(18))

##### Using function to clean data


You can define a function with following syntax

```
def function_name():
    funciton_expression
```

In [ ]:
def hello_world(name):
    return "Hello, " + name

print(hello_world('Will'))
print(hello_world('Taiwan'))

You can write custom function and use `df.apply()` to make it work in DataFrame.

In [ ]:
import numpy as np
import pandas as pd

item_price_df = pd.DataFrame(np.array([['Hot Dog', 30], ['bread', 45]]))
item_price_df.columns = ['item', 'price']

print(item_price_df)

def add_currency(price, currency=None):
    return currency + str(price)

item_price_df['price_with_currency'] = item_price_df.price.apply(add_currency, currency='GBP')

print(item_price_df)

##### Using Lambda function to clean data

`Lambda function` is a Python powerful feature which could help you to iterate over list.

In [ ]:
print(item_price_df)

currency = 'NTD'
item_price_df['price_with_currency_2'] = item_price_df.price.apply(lambda x: currency + str(x))

print(item_price_df)

##### Duplicate and Missing Value

You can call `df.drop_duplicates()` to drop duplicates.

In [ ]:
import numpy as np
import pandas as pd

dup_df = pd.DataFrame(np.array([['dog'], ['dog'], ['cat']]))
dup_df.columns = ['animal']

print(dup_df)

dup_df = dup_df.drop_duplicates()

print(dup_df)

You can call `df.dropna()` to remove rows with missing data.

In [ ]:
import pandas as pd

pm_2_5_file_path = './res/observation_pm_2_5.csv'

# Some rows has value 'NR', we treat it as null
df = pd.read_csv(pm_2_5_file_path, na_values='NR')

print(df.info())

drop_na_df = df.dropna()

print(drop_na_df.info())

You can also fill missing value with other value using `df.fillna(value)`, such as average.

In [ ]:
means = df.mean()

fill_na_df = df.copy()
print(fill_na_df.info())

for i in range(0, 24):
    print('mean of hour ' + str(i) + ' is ' + str(means[i]))
    # i+2 because column has Date and Observation Item columns
    fill_na_df[str(i)] = fill_na_df[str(i)].fillna(means[i])
    
print(fill_na_df.info())

##### Assert data

You can use `assert` syntax to make sure your data is correct

In [ ]:
assert 1 == 1

assert 1 == 0

You can use `pd.notnull(df)` to check if it contains null value

In [ ]:
import pandas as pd

assert pd.notnull(fill_na_df).all().all()

assert pd.notnull(df).all().all()

### Q: Now, you can play DataFrame `life_expectancy_df` whatever you like. such as show line chart of life expectancy in specific country. 

In [ ]:
# Write your code here
